In [102]:
from statsbombpy import sb
import pandas as pd
from tqdm import tqdm
tqdm.pandas()
# USE STATSBOMB LOGO IN REPORT!!!

# Ignore NoAuthWarning, since we are not using an API key
import warnings
warnings.filterwarnings('ignore', category=sb.api_client.NoAuthWarning)

In [103]:
sb.competitions().head()

,competition_id,season_id,country_name,competition_name,competition_gender,competition_youth,competition_international,season_name,match_updated,match_updated_360,match_available_360,match_available
0,9,27,Germany,1. Bundesliga,male,False,False,2015/2016,2023-08-17T23:51:11.837478,None,None,2023-08-17T23:51:11.837478
1,16,4,Europe,Champions League,male,False,False,2018/2019,2023-03-07T12:20:48.118250,2021-06-13T16:17:31.694,None,2023-03-07T12:20:48.118250
2,16,1,Europe,Champions League,male,False,False,2017/2018,2021-08-27T11:26:39.802832,2021-06-13T16:17:31.694,None,2021-01-23T21:55:30.425330
3,16,2,Europe,Champions League,male,False,False,2016/2017,2021-08-27T11:26:39.802832,2021-06-13T16:17:31.694,None,2020-07-29T05:00
4,16,27,Europe,Champions League,male,False,False,2015/2016,2021-08-27T11:26:39.802832,2021-06-13T16:17:31.694,None,2020-07-29T05:00


In [104]:
SEASON_CUTOFF = 2015 # Last season to be included
all_season_to_look_at = list(range(SEASON_CUTOFF, 2025))

from itertools import product # Method to get the cartesian product
# All possible season strings in that time period
all_season_to_look_at = list(map(lambda x: f'{x[0]}/{x[1]}' if x[0] != x[1] else str(x[0]),filter(lambda x: x[0] == x[1] or x[1] - x[0] == 1, product(all_season_to_look_at, all_season_to_look_at))))
all_season_to_look_at

['2015',
 '2015/2016',
 '2016',
 '2016/2017',
 '2017',
 '2017/2018',
 '2018',
 '2018/2019',
 '2019',
 '2019/2020',
 '2020',
 '2020/2021',
 '2021',
 '2021/2022',
 '2022',
 '2022/2023',
 '2023',
 '2023/2024',
 '2024']

In [105]:
# Select all seasons up until cutoff
all_relevant_competitions = sb.competitions()[sb.competitions()['season_name'].isin(all_season_to_look_at)].copy()
all_relevant_competitions.head()

,competition_id,season_id,country_name,competition_name,competition_gender,competition_youth,competition_international,season_name,match_updated,match_updated_360,match_available_360,match_available
0,9,27,Germany,1. Bundesliga,male,False,False,2015/2016,2023-08-17T23:51:11.837478,None,None,2023-08-17T23:51:11.837478
1,16,4,Europe,Champions League,male,False,False,2018/2019,2023-03-07T12:20:48.118250,2021-06-13T16:17:31.694,None,2023-03-07T12:20:48.118250
2,16,1,Europe,Champions League,male,False,False,2017/2018,2021-08-27T11:26:39.802832,2021-06-13T16:17:31.694,None,2021-01-23T21:55:30.425330
3,16,2,Europe,Champions League,male,False,False,2016/2017,2021-08-27T11:26:39.802832,2021-06-13T16:17:31.694,None,2020-07-29T05:00
4,16,27,Europe,Champions League,male,False,False,2015/2016,2021-08-27T11:26:39.802832,2021-06-13T16:17:31.694,None,2020-07-29T05:00


In [106]:
# Gather all relevant match_ids
all_relevant_matches = []
for competition_id in all_relevant_competitions['competition_id'].unique():
    for season_id in all_relevant_competitions[all_relevant_competitions['competition_id'] == competition_id]['season_id'].unique():
        all_relevant_matches.append(sb.matches(competition_id = competition_id, season_id = season_id).copy())
# Concatenate all matches
all_relevant_matches = pd.concat(all_relevant_matches, ignore_index = True)
all_relevant_matches.head()

,match_id,match_date,kick_off,competition,season,home_team,away_team,home_score,away_score,match_status,...,last_updated_360,match_week,competition_stage,stadium,referee,home_managers,away_managers,data_version,shot_fidelity_version,xy_fidelity_version
0,3890561,2016-05-14,15:30:00.000,Germany - 1. Bundesliga,2015/2016,Hoffenheim,Schalke 04,1,4,available,...,None,34,Regular Season,PreZero Arena,Felix Brych,Julian Nagelsmann,André Breitenreiter,1.1.0,2,2
1,3890505,2016-04-02,15:30:00.000,Germany - 1. Bundesliga,2015/2016,Bayern Munich,Eintracht Frankfurt,1,0,available,...,None,28,Regular Season,Allianz Arena,Florian Meyer,Josep Guardiola i Sala,Niko Kovač,1.1.0,2,2
2,3890511,2016-04-08,20:30:00.000,Germany - 1. Bundesliga,2015/2016,Hertha Berlin,Hannover 96,2,2,available,...,None,29,Regular Season,Olympiastadion Berlin,Benjamin Brand,Pál Dárdai,Daniel Stendel,1.1.0,2,2
3,3890515,2016-04-09,15:30:00.000,Germany - 1. Bundesliga,2015/2016,Hamburger SV,Darmstadt 98,1,2,available,...,None,29,Regular Season,Volksparkstadion,Peter Sippel,Bruno Labbadia,Dirk Schuster,1.1.0,2,2
4,3890411,2015-12-20,16:30:00.000,Germany - 1. Bundesliga,2015/2016,Hertha Berlin,FSV Mainz 05,2,0,available,...,None,17,Regular Season,Olympiastadion Berlin,Peter Sippel,Pál Dárdai,Martin Schmidt,1.1.0,2,2


In [107]:
# Save all relevant matches to a csv file
all_relevant_matches.to_csv(f'all_relevant_matches_lower_bound_{SEASON_CUTOFF}.csv', index = False)

In [108]:
# Free up some memory
del all_relevant_competitions
del all_season_to_look_at

In [109]:
# A function to get all the shot events from a match as a dataframe
def get_shot_events(match_id):
    # Get all events from the match
    match_events = sb.events(match_id = match_id)
    # Create boolean series to filter out all shot events with xG, with xg2 if existent
    xg_shot_events = (match_events['shot_statsbomb_xg'].notnull() | match_events['shot_statsbomb_xg2'].notnull()) if 'shot_statsbomb_xg2' in match_events.columns else match_events['shot_statsbomb_xg'].notnull()
    # Return all shot events with xG
    return match_events[xg_shot_events].copy()

In [110]:
# # Create a dataframe with all shot events which have xG from all matches
# all_xg_data = pd.concat(all_relevant_matches['match_id'].copy().progress_apply(get_shot_events), ignore_index = True)
# all_xg_data.info()

In [111]:
get_shot_events(3890561)

,ball_receipt_outcome,ball_recovery_recovery_failure,block_deflection,carry_end_location,clearance_aerial_won,clearance_body_part,clearance_head,clearance_left_foot,clearance_right_foot,counterpress,...,shot_technique,shot_type,substitution_outcome,substitution_replacement,tactics,team,team_id,timestamp,type,under_pressure
3692,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Half Volley,Open Play,NaN,NaN,NaN,Hoffenheim,175,00:04:21.052,Shot,NaN
3693,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Volley,Open Play,NaN,NaN,NaN,Schalke 04,181,00:06:27.395,Shot,NaN
3694,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Half Volley,Open Play,NaN,NaN,NaN,Hoffenheim,175,00:08:16.762,Shot,True
3695,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Normal,Open Play,NaN,NaN,NaN,Schalke 04,181,00:13:55.721,Shot,NaN
3696,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Normal,Open Play,NaN,NaN,NaN,Schalke 04,181,00:17:16.953,Shot,NaN
3697,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Normal,Open Play,NaN,NaN,NaN,Schalke 04,181,00:24:26.914,Shot,NaN
3698,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Normal,Open Play,NaN,NaN,NaN,Schalke 04,181,00:25:04.076,Shot,True
3699,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Normal,Open Play,NaN,NaN,NaN,Schalke 04,181,00:39:00.667,Shot,NaN
3700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Half Volley,Open Play,NaN,NaN,NaN,Hoffenheim,175,00:40:06.160,Shot,NaN
3701,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Normal,Open Play,NaN,NaN,NaN,Hoffenheim,175,00:44:33.291,Shot,NaN
